In [1]:
from keras import backend as K
import keras
import tensorflow as tf
#from keras.layers import DepthwiseConv2D, ReLU
from keras.utils.generic_utils import CustomObjectScope
from keras.layers import Dense, Activation, Flatten#, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.models import load_model
#from keras_applications import mobilenet
from keras.applications.mobilenet import MobileNet

# This line must be executed before loading Keras model.
K.set_learning_phase(0)

with CustomObjectScope({'relu6': keras.layers.ReLU(6.)}):
    model = load_model('./models/mobileTrain.h5', custom_objects ={'relu6': keras.layers.ReLU(6.)})
    

# model = load_model('./models/mobileTrain.h5')
print(model.outputs)
# [<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>]
print(model.inputs)
# [<tf.Tensor 'conv2d_1_input:0' shape=(?, 28, 28, 1) dtype=float32>]

from keras import backend as K
import tensorflow as tf

output_names=[out.op.name for out in model.outputs]
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])


tf.train.write_graph(frozen_graph, "models", "TF_mobileTrain.pb", as_text=False)

# from tensorflow.compiler.tensorrt import trt_convert as trt
# converter = trt.TrtGraphConverter(
#     input_graph_def=frozen_graph,
#     nodes_blacklist=['logits', 'classes'])
# TRT_frozen_graph = converter.convert()

# tf.train.write_graph(TRT_frozen_graph, "models", "TRT_mobileTrain.pb", as_text=False)

import tensorflow.contrib.tensorrt as trt

TRT_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph,
    outputs=output_names,
    max_batch_size=1,
    max_workspace_size_bytes=1 << 25,
    precision_mode='FP32',
    minimum_segment_size=50
)

from tensorflow.python.framework import graph_io

graph_io.write_graph(TRT_graph, "./models/","TRT_graph.pb", as_text=False)

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:522: FutureWarn

[<tf.Tensor 'dense_2/BiasAdd:0' shape=(?, 1) dtype=float32>]
[<tf.Tensor 'input_1:0' shape=(?, 224, 224, 3) dtype=float32>]
